In [ ]:
# Ensure nested async loops work in Jupyter
import nest_asyncio
nest_asyncio.apply()

# Load environment variables from .env
from dotenv import load_dotenv
load_dotenv()

# Imports
import os
import sys
import time
from typing import List, Dict, Any
from IPython.display import display, Markdown


notebook_path = os.getcwd()
root_path = os.path.abspath(os.path.join(notebook_path, '..', '..', '..'))

# Add root path to system path so Python can find `llm/config.py`
sys.path.append(root_path)

# Now import the OpenAI key from your config
try:
    from llm.config import openai_key
except ImportError:
    display(Markdown("**❌ Error:** Could not import `openai_key` from `llm/config.py`"))
    raise

# Initialize OpenAI client
from openai import OpenAI
client = OpenAI(api_key=openai_key)

# Confirmation display
display(Markdown("✅ **OpenAI client initialized successfully**"))


✅ **OpenAI client initialized successfully**

In [2]:
import ipywidgets as widgets 
from IPython.display import display, Markdown

In [3]:
sample_text = (
    "The new product launch exceeded expectations, with sales reaching 150% of the Q1 target. Key contributing factors included the revamped marketing strategy and strong positive customer reviews. However, supply chain bottlenecks led to minor delays for 10% of orders, an issue now being addressed by diversifying suppliers. The financial outlook for Q2 remains robust."
)

prompts = {
    "Summarizer_Simple": "Summarize this text: {text_input}",
    "Summarizer_Executive": (
        "You are an expert summarizer for busy executives. Summarize this for quick understanding, focusing on key decisions and outcomes: {text_input}"
    )
}


In [4]:
if "prompt_selector" not in globals():
    prompt_selector = widgets.Dropdown(options=["Summarizer_Simple", "Summarizer_Executive"], description='Prompt:')
    temp_slider = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.1, description='Temp:')
    run_button = widgets.Button(description="Run Prompt")
    output_area = widgets.Output()

In [5]:
def run_selected_prompt_sync(b):
    output_area.clear_output(wait=True)
    selected_prompt_name = prompt_selector.value
    temperature = temp_slider.value
    template = prompts[selected_prompt_name]
    prompt = template.format(text_input=sample_text)

    # Use append_display_data inside output_area context
    with output_area:
        output_area.append_display_data(Markdown(f"**Running '{selected_prompt_name}' with temp={temperature}...**"))
        try:
            start = time.perf_counter()
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": prompt}],
                temperature=temperature,
            )
            latency = time.perf_counter() - start
            content = response.choices[0].message.content

            output_area.append_display_data(Markdown(f"**Response:**\n```\n{content}\n```"))
            output_area.append_display_data(Markdown(f"*Latency: {latency:.2f}s, Tokens: {response.usage.total_tokens}*"))

        except Exception as e:
            output_area.append_display_data(Markdown(f"**Error:** {e}"))


In [6]:
# Unbind previous click handlers to avoid duplicate executions
run_button._click_handlers.callbacks = []

# Re-bind the button to the handler
run_button.on_click(run_selected_prompt_sync)

# Display widgets
display(prompt_selector, temp_slider, run_button, output_area)


Dropdown(description='Prompt:', options=('Summarizer_Simple', 'Summarizer_Executive'), value='Summarizer_Simpl…

FloatSlider(value=0.5, description='Temp:', max=1.0)

Button(description='Run Prompt', style=ButtonStyle())

Output()